### **Основное задание**



Задание.  
- Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) которые бы задавались следующим образом:  
    - x - последовательность цифр  
    - y1 = x1, y(i) = x(i) + x(1). Если y(i)>=10, то y(i)=y(i)-10
-  Задача научить модель предсказывать y(i) по x(i)
- Опробовать RNN, LSTM, GRU


# Загружаем библиотеки. Смотрим, что доступно cpu или cuda. Загружаем данные.  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import random
import time
import numpy as np

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cpu')

In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/RNN_LSTM_GRU/'


# Сгенерируем последовательности.

Сгенерируем 2000 последовательностей, возьмём длину 30.  
В качестве предсказываемой последовательности возьмём пока нули.    

In [ ]:
X = np.random.randint(10, size=(2000, 30), dtype=int)
y = np.zeros((2000, 30), dtype=int)

In [ ]:
X[:5]

array([[0, 9, 7, 4, 3, 1, 9, 5, 1, 4, 4, 2, 6, 5, 7, 3, 6, 0, 9, 9, 3, 4,
        5, 3, 1, 2, 6, 0, 3, 5],
       [7, 1, 4, 0, 7, 6, 4, 8, 0, 4, 2, 7, 6, 7, 9, 4, 7, 2, 1, 6, 4, 5,
        9, 3, 5, 6, 7, 0, 7, 8],
       [3, 7, 0, 2, 4, 4, 3, 4, 0, 2, 2, 5, 2, 1, 8, 5, 0, 5, 8, 0, 6, 6,
        7, 3, 7, 0, 7, 4, 9, 5],
       [9, 3, 3, 0, 7, 9, 7, 8, 9, 7, 0, 1, 5, 0, 6, 5, 6, 7, 2, 4, 1, 0,
        5, 2, 9, 5, 2, 5, 1, 7],
       [5, 7, 9, 4, 4, 2, 5, 9, 1, 8, 4, 4, 9, 3, 1, 6, 6, 9, 9, 1, 9, 4,
        6, 3, 0, 4, 2, 0, 2, 9]])

Сгенерируем значения y, согласно условию задачи. 

In [ ]:
for i in range(2000):
    y[i][0] = X[i][0]
    for j in range(1, 30):
        y[i][j] = (X[i][j] + X[i][0]) % 10

In [ ]:
y[:5]

array([[0, 9, 7, 4, 3, 1, 9, 5, 1, 4, 4, 2, 6, 5, 7, 3, 6, 0, 9, 9, 3, 4,
        5, 3, 1, 2, 6, 0, 3, 5],
       [7, 8, 1, 7, 4, 3, 1, 5, 7, 1, 9, 4, 3, 4, 6, 1, 4, 9, 8, 3, 1, 2,
        6, 0, 2, 3, 4, 7, 4, 5],
       [3, 0, 3, 5, 7, 7, 6, 7, 3, 5, 5, 8, 5, 4, 1, 8, 3, 8, 1, 3, 9, 9,
        0, 6, 0, 3, 0, 7, 2, 8],
       [9, 2, 2, 9, 6, 8, 6, 7, 8, 6, 9, 0, 4, 9, 5, 4, 5, 6, 1, 3, 0, 9,
        4, 1, 8, 4, 1, 4, 0, 6],
       [5, 2, 4, 9, 9, 7, 0, 4, 6, 3, 9, 9, 4, 8, 6, 1, 1, 4, 4, 6, 4, 9,
        1, 8, 5, 9, 7, 5, 7, 4]])

Возьмём в качестве y один из столбцов (последний).

In [ ]:
y = y[:,-1]

In [ ]:
y

array([5, 5, 8, ..., 6, 6, 4])

Перейдём из numpy в torch.

In [ ]:
X, y = torch.from_numpy(X), torch.from_numpy(y)

Создадим датасет из наших данных и загрузим в Loader.

In [ ]:
batch_size = 128

numbers_dataset = TensorDataset(X, y)
numbers_loader = torch.utils.data.DataLoader(numbers_dataset, batch_size,
                                             shuffle=True)

# **Строим нейронную сеть**

Создадим класс для всех типов ячеек (RNN, LSTM, GRU).



In [ ]:
class RecurrentNetwork(torch.nn.Module):
    def __init__(self, rnn, vocab_size, embedding_dim, hidden_dim, n_classes):
        super(RecurrentNetwork, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = rnn(embedding_dim, hidden_dim, batch_first=True)
        self.dense = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, X):
        out = self.embedding(X)
        _, state = self.rnn(out)
        pred = self.dense(state[0])
        if isinstance(self.rnn, torch.nn.LSTM):
            pred = pred.squeeze(0)
        return pred

Функция обучения.

In [ ]:
def train_model(model, loader, criterion, optimizer, n_epochs=100,):
    train_losses = []

    for epoch in tqdm(range(1, n_epochs + 1)):
        start = time.time()
        train_loss = 0.0
        model.train()

        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            y_pred = model.forward(X_batch)
            loss = criterion(y_pred, y_batch)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(loader)
        train_losses.append(train_loss)
        sec = time.time() - start
        stat = f'Epoch: {epoch}, time: {sec:.1f}s, loss: {train_loss:.3f}'
        print(stat)

        
    return model, train_losses

## **RNN**

## vocab_size 10

In [ ]:
vocab_size = 10
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.RNN, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 0.6s, loss: 2.321
Epoch: 2, time: 0.4s, loss: 2.301
Epoch: 3, time: 0.4s, loss: 2.289
Epoch: 4, time: 0.4s, loss: 2.281
Epoch: 5, time: 0.3s, loss: 2.275
Epoch: 6, time: 0.4s, loss: 2.271
Epoch: 7, time: 0.4s, loss: 2.268
Epoch: 8, time: 0.4s, loss: 2.265
Epoch: 9, time: 0.4s, loss: 2.263
Epoch: 10, time: 0.4s, loss: 2.259
Epoch: 11, time: 0.3s, loss: 2.257
Epoch: 12, time: 0.4s, loss: 2.255
Epoch: 13, time: 0.3s, loss: 2.253
Epoch: 14, time: 0.4s, loss: 2.250
Epoch: 15, time: 0.3s, loss: 2.248
Epoch: 16, time: 0.4s, loss: 2.245
Epoch: 17, time: 0.3s, loss: 2.244
Epoch: 18, time: 0.4s, loss: 2.242
Epoch: 19, time: 0.3s, loss: 2.240
Epoch: 20, time: 0.3s, loss: 2.238
Epoch: 21, time: 0.4s, loss: 2.237
Epoch: 22, time: 0.4s, loss: 2.235
Epoch: 23, time: 0.3s, loss: 2.233
Epoch: 24, time: 0.4s, loss: 2.230
Epoch: 25, time: 0.4s, loss: 2.230
Epoch: 26, time: 0.4s, loss: 2.226
Epoch: 27, time: 0.4s, loss: 2.224
Epoch: 28, time: 0.3s, loss: 2.222
Epoch: 29, time: 0.4s, loss: 

## vocab_size = 20

In [ ]:
vocab_size = 20
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.RNN, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 0.4s, loss: 2.898
Epoch: 2, time: 0.4s, loss: 2.647
Epoch: 3, time: 0.4s, loss: 2.432
Epoch: 4, time: 0.4s, loss: 2.356
Epoch: 5, time: 0.4s, loss: 2.328
Epoch: 6, time: 0.4s, loss: 2.315
Epoch: 7, time: 0.4s, loss: 2.308
Epoch: 8, time: 0.3s, loss: 2.301
Epoch: 9, time: 0.3s, loss: 2.297
Epoch: 10, time: 0.4s, loss: 2.296
Epoch: 11, time: 0.4s, loss: 2.291
Epoch: 12, time: 0.4s, loss: 2.291
Epoch: 13, time: 0.4s, loss: 2.286
Epoch: 14, time: 0.4s, loss: 2.284
Epoch: 15, time: 0.4s, loss: 2.285
Epoch: 16, time: 0.4s, loss: 2.282
Epoch: 17, time: 0.4s, loss: 2.281
Epoch: 18, time: 0.4s, loss: 2.280
Epoch: 19, time: 0.4s, loss: 2.278
Epoch: 20, time: 0.4s, loss: 2.276
Epoch: 21, time: 0.4s, loss: 2.277
Epoch: 22, time: 0.4s, loss: 2.271
Epoch: 23, time: 0.4s, loss: 2.273
Epoch: 24, time: 0.4s, loss: 2.272
Epoch: 25, time: 0.4s, loss: 2.268
Epoch: 26, time: 0.3s, loss: 2.268
Epoch: 27, time: 0.4s, loss: 2.267
Epoch: 28, time: 0.4s, loss: 2.265
Epoch: 29, time: 0.4s, loss: 

### Наблюдение

Увеличение vocab_size приводит к замедлению обучения.

## **LSTM**

## vocab_size = 10

In [ ]:
vocab_size = 10
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.LSTM, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 1.2s, loss: 2.307
Epoch: 2, time: 1.1s, loss: 2.304
Epoch: 3, time: 1.1s, loss: 2.302
Epoch: 4, time: 1.1s, loss: 2.301
Epoch: 5, time: 1.1s, loss: 2.300
Epoch: 6, time: 1.1s, loss: 2.298
Epoch: 7, time: 1.1s, loss: 2.297
Epoch: 8, time: 1.1s, loss: 2.296
Epoch: 9, time: 1.0s, loss: 2.295
Epoch: 10, time: 1.1s, loss: 2.294
Epoch: 11, time: 1.0s, loss: 2.293
Epoch: 12, time: 1.1s, loss: 2.292
Epoch: 13, time: 1.1s, loss: 2.291
Epoch: 14, time: 1.1s, loss: 2.291
Epoch: 15, time: 1.1s, loss: 2.290
Epoch: 16, time: 1.1s, loss: 2.290
Epoch: 17, time: 1.1s, loss: 2.289
Epoch: 18, time: 1.1s, loss: 2.288
Epoch: 19, time: 1.0s, loss: 2.288
Epoch: 20, time: 1.1s, loss: 2.287
Epoch: 21, time: 1.0s, loss: 2.286
Epoch: 22, time: 1.1s, loss: 2.285
Epoch: 23, time: 1.1s, loss: 2.285
Epoch: 24, time: 1.1s, loss: 2.285
Epoch: 25, time: 1.1s, loss: 2.284
Epoch: 26, time: 1.1s, loss: 2.284
Epoch: 27, time: 1.1s, loss: 2.283
Epoch: 28, time: 1.1s, loss: 2.283
Epoch: 29, time: 1.1s, loss: 

Следует обучать дальше в будущем. Видно, что за 1000 эпох обучилось только до 0.005 в отличии от RNN.


## vocab_size = 20

In [ ]:
vocab_size = 20
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.LSTM, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 1.0s, loss: 2.971
Epoch: 2, time: 1.0s, loss: 2.894
Epoch: 3, time: 1.0s, loss: 2.819
Epoch: 4, time: 1.0s, loss: 2.745
Epoch: 5, time: 1.0s, loss: 2.673
Epoch: 6, time: 1.0s, loss: 2.604
Epoch: 7, time: 1.0s, loss: 2.544
Epoch: 8, time: 1.0s, loss: 2.494
Epoch: 9, time: 1.0s, loss: 2.454
Epoch: 10, time: 1.0s, loss: 2.422
Epoch: 11, time: 1.0s, loss: 2.397
Epoch: 12, time: 1.0s, loss: 2.379
Epoch: 13, time: 1.0s, loss: 2.365
Epoch: 14, time: 1.1s, loss: 2.354
Epoch: 15, time: 1.0s, loss: 2.345
Epoch: 16, time: 1.0s, loss: 2.337
Epoch: 17, time: 1.1s, loss: 2.331
Epoch: 18, time: 1.0s, loss: 2.327
Epoch: 19, time: 1.1s, loss: 2.322
Epoch: 20, time: 1.0s, loss: 2.319
Epoch: 21, time: 1.0s, loss: 2.316
Epoch: 22, time: 1.0s, loss: 2.313
Epoch: 23, time: 1.0s, loss: 2.311
Epoch: 24, time: 1.0s, loss: 2.309
Epoch: 25, time: 1.0s, loss: 2.307
Epoch: 26, time: 1.0s, loss: 2.305
Epoch: 27, time: 1.1s, loss: 2.303
Epoch: 28, time: 1.0s, loss: 2.303
Epoch: 29, time: 1.0s, loss: 

А вот LSTM при увеличении длины лучше обучилась.

## Наблюдение.

Замена RNN на LSTM хороша для больших длин.

## **GRU**

## vocab_size = 10

In [ ]:
vocab_size = 10
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.GRU, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 0.9s, loss: 2.926
Epoch: 2, time: 0.9s, loss: 2.722
Epoch: 3, time: 0.9s, loss: 2.558
Epoch: 4, time: 0.9s, loss: 2.453
Epoch: 5, time: 0.9s, loss: 2.395
Epoch: 6, time: 0.9s, loss: 2.362
Epoch: 7, time: 0.9s, loss: 2.343
Epoch: 8, time: 0.9s, loss: 2.331
Epoch: 9, time: 0.9s, loss: 2.321
Epoch: 10, time: 0.9s, loss: 2.314
Epoch: 11, time: 0.9s, loss: 2.311
Epoch: 12, time: 0.9s, loss: 2.306
Epoch: 13, time: 0.9s, loss: 2.305
Epoch: 14, time: 0.9s, loss: 2.300
Epoch: 15, time: 0.9s, loss: 2.299
Epoch: 16, time: 0.9s, loss: 2.297
Epoch: 17, time: 0.9s, loss: 2.296
Epoch: 18, time: 0.9s, loss: 2.294
Epoch: 19, time: 0.9s, loss: 2.292
Epoch: 20, time: 0.9s, loss: 2.293
Epoch: 21, time: 0.9s, loss: 2.291
Epoch: 22, time: 0.9s, loss: 2.290
Epoch: 23, time: 0.9s, loss: 2.289
Epoch: 24, time: 0.9s, loss: 2.289
Epoch: 25, time: 0.9s, loss: 2.288
Epoch: 26, time: 0.9s, loss: 2.289
Epoch: 27, time: 0.9s, loss: 2.286
Epoch: 28, time: 0.9s, loss: 2.287
Epoch: 29, time: 0.9s, loss: 

Медленней обучается по сравнению с RNN и LSTM.

## vocab_size = 20

In [ ]:
vocab_size = 20
embedding_dim = 64
hidden_dim = 128

In [ ]:
model = RecurrentNetwork(
    torch.nn.GRU, vocab_size, embedding_dim, hidden_dim, 10)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [ ]:
model, train_losses = train_model(model, numbers_loader, criterion, optimizer, n_epochs=1000)

Epoch: 1, time: 0.9s, loss: 2.936
Epoch: 2, time: 0.9s, loss: 2.749
Epoch: 3, time: 0.9s, loss: 2.590
Epoch: 4, time: 0.9s, loss: 2.476
Epoch: 5, time: 0.9s, loss: 2.409
Epoch: 6, time: 0.9s, loss: 2.370
Epoch: 7, time: 0.9s, loss: 2.348
Epoch: 8, time: 0.9s, loss: 2.332
Epoch: 9, time: 0.9s, loss: 2.323
Epoch: 10, time: 0.9s, loss: 2.317
Epoch: 11, time: 0.9s, loss: 2.312
Epoch: 12, time: 0.9s, loss: 2.308
Epoch: 13, time: 0.9s, loss: 2.305
Epoch: 14, time: 0.9s, loss: 2.301
Epoch: 15, time: 0.9s, loss: 2.300
Epoch: 16, time: 0.9s, loss: 2.297
Epoch: 17, time: 0.9s, loss: 2.296
Epoch: 18, time: 0.9s, loss: 2.295
Epoch: 19, time: 0.9s, loss: 2.293
Epoch: 20, time: 0.9s, loss: 2.293
Epoch: 21, time: 0.9s, loss: 2.293
Epoch: 22, time: 0.9s, loss: 2.290
Epoch: 23, time: 0.9s, loss: 2.289
Epoch: 24, time: 0.9s, loss: 2.288
Epoch: 25, time: 0.9s, loss: 2.287
Epoch: 26, time: 0.9s, loss: 2.288
Epoch: 27, time: 0.9s, loss: 2.287
Epoch: 28, time: 0.9s, loss: 2.286
Epoch: 29, time: 0.9s, loss: 

С увеличением длины медленней обучается.